# SQLite 7 Databricks (Manual Upload 7 COPY INTO from Unity Catalog Volume)

This notebook loads CSVs you manually uploaded into a Unity Catalog **Volume** into Delta tables using `COPY INTO`.

## Assumptions
- You uploaded CSV files to this Volume folder (one file per table):
  - `alerts.csv`, `analysis_runs.csv`, `backtest_results.csv`, `backtest_trades.csv`, `market_cap_cache.csv`, `ohlc.csv`, `snapshot_cache.csv`, `trade_plans.csv`
- Upload target folder (Volume path): `/Volumes/workspace/squeeze/squeeze_bronze`
- CSVs have a header row and are quoted (works with the `FORMAT_OPTIONS` below).

If you used a different catalog/schema/volume name, update the config cell.

In [ ]:
# ---- CONFIG ----
catalog = "workspace"
schema = "squeeze"
# Use the DBFS-style path for Volumes
base_dir = "dbfs:/Volumes/workspace/squeeze/squeeze_bronze"

tables = [
    "alerts",
    "analysis_runs",
    "backtest_results",
    "backtest_trades",
    "market_cap_cache",
    "ohlc",
    "snapshot_cache",
    "trade_plans",
]

print("Target: ", f"{catalog}.{schema}")
print("Source dir:", base_dir)


In [ ]:
# Optional: confirm the files are visible
display(dbutils.fs.ls(base_dir))

In [ ]:
%sql
-- Create and select namespace
-- NOTE: In many workspaces you cannot CREATE CATALOG; usually the catalog already exists.
USE CATALOG `workspace`;
CREATE SCHEMA IF NOT EXISTS `squeeze`;
USE SCHEMA `squeeze`;


## Create Delta tables
These DDLs mirror the repo ETL type mapping (SQLite INT, BIGINT, REAL, DOUBLE, TEXT, STRING).
If any table differs from your actual SQLite schema, adjust as needed.

In [ ]:
%sql
USE CATALOG `workspace`;
USE SCHEMA `squeeze`;

CREATE TABLE IF NOT EXISTS alerts (
  `id` BIGINT,
  `ts` BIGINT,
  `created_ts` BIGINT,
  `exchange` STRING,
  `symbol` STRING,
  `signal` STRING,
  `source_tf` STRING,
  `price` DOUBLE,
  `reason` STRING,
  `setup_score` DOUBLE,
  `setup_grade` STRING,
  `avoid_reasons` STRING,
  `metrics_json` STRING
) USING DELTA;

CREATE TABLE IF NOT EXISTS analysis_runs (
  `id` BIGINT,
  `ts` BIGINT,
  `window_days` BIGINT,
  `exchange` STRING,
  `top200_only` BIGINT,
  `n_alerts` BIGINT
) USING DELTA;

CREATE TABLE IF NOT EXISTS backtest_results (
  `id` BIGINT,
  `ts` BIGINT,
  `exchange` STRING,
  `symbol` STRING,
  `source_tf` STRING,
  `window_days` BIGINT,
  `strategy_version` STRING,
  `n_trades` BIGINT,
  `win_rate` DOUBLE,
  `avg_return` DOUBLE,
  `total_return` DOUBLE,
  `max_drawdown` DOUBLE,
  `notes` STRING
) USING DELTA;

CREATE TABLE IF NOT EXISTS backtest_trades (
  `id` BIGINT,
  `backtest_id` BIGINT,
  `ts` BIGINT,
  `exchange` STRING,
  `symbol` STRING,
  `source_tf` STRING,
  `entry_ts` BIGINT,
  `exit_ts` BIGINT,
  `entry_price` DOUBLE,
  `exit_price` DOUBLE,
  `return_pct` DOUBLE,
  `hold_minutes` BIGINT,
  `meta_json` STRING
) USING DELTA;

CREATE TABLE IF NOT EXISTS market_cap_cache (
  `id` BIGINT,
  `ts` BIGINT,
  `symbol` STRING,
  `response_json` STRING
) USING DELTA;

CREATE TABLE IF NOT EXISTS ohlc (
  `id` BIGINT,
  `exchange` STRING,
  `symbol` STRING,
  `interval` STRING,
  `open_time` BIGINT,
  `close_time` BIGINT,
  `open` DOUBLE,
  `high` DOUBLE,
  `low` DOUBLE,
  `close` DOUBLE,
  `volume` DOUBLE
) USING DELTA;

CREATE TABLE IF NOT EXISTS snapshot_cache (
  `id` BIGINT,
  `exchange` STRING,
  `ts` BIGINT,
  `snapshot_json` STRING
) USING DELTA;

CREATE TABLE IF NOT EXISTS trade_plans (
  `id` BIGINT,
  `ts` BIGINT,
  `created_ts` BIGINT,
  `exchange` STRING,
  `symbol` STRING,
  `plan_json` STRING
) USING DELTA;


## Optional: Truncate tables (full refresh)
Uncomment if you want to replace table contents each run.

In [ ]:
%sql
-- USE CATALOG `workspace`;
-- USE SCHEMA `squeeze`;
--
-- TRUNCATE TABLE alerts;
-- TRUNCATE TABLE analysis_runs;
-- TRUNCATE TABLE backtest_results;
-- TRUNCATE TABLE backtest_trades;
-- TRUNCATE TABLE market_cap_cache;
-- TRUNCATE TABLE ohlc;
-- TRUNCATE TABLE snapshot_cache;
-- TRUNCATE TABLE trade_plans;


In [ ]:
# Run COPY INTO for each table
def copy_into(table: str):
    src = f"{base_dir}/{table}.csv"
    full_table = f"`{catalog}`.`{schema}`.`{table}`"
    sql = f"""
COPY INTO {full_table}
FROM '{src}'
FILEFORMAT = CSV
FORMAT_OPTIONS (
  'header' = 'true',
  'quote' = '\',
  'escape' = '\\\\',
  'multiLine' = 'true'
)
"""
    print(f"[copy-into] {table} <= {src}")
    spark.sql(sql)

for t in tables:
    copy_into(t)


In [ ]:
%sql
-- Validate row counts
USE CATALOG `workspace`;
USE SCHEMA `squeeze`;

SELECT 'alerts' AS table, COUNT(*) AS n FROM alerts
UNION ALL SELECT 'analysis_runs', COUNT(*) FROM analysis_runs
UNION ALL SELECT 'backtest_results', COUNT(*) FROM backtest_results
UNION ALL SELECT 'backtest_trades', COUNT(*) FROM backtest_trades
UNION ALL SELECT 'market_cap_cache', COUNT(*) FROM market_cap_cache
UNION ALL SELECT 'ohlc', COUNT(*) FROM ohlc
UNION ALL SELECT 'snapshot_cache', COUNT(*) FROM snapshot_cache
UNION ALL SELECT 'trade_plans', COUNT(*) FROM trade_plans
ORDER BY table;
